In [12]:
import subprocess
import os
from LAMMPS_gen_in_file import * 
from LAMMPS_data_preprocess import * 
from LAMMPS_analysis import *
from concurrent.futures import ProcessPoolExecutor
import pandas as pd

In [13]:
def run_lammps_simulation(velocity, temperature):
    dump_dir = "./runs/test_" + str(test_num) + "/vel_" + str(velocity) + "_temp_" + str(temperature)
    dump_dir_ = "runs/test_" + str(test_num) + "/vel_" + str(velocity) + "_temp_" + str(temperature)
    
    temp_ts = 0.03 * 0.02 / velocity
    
    try:
        os.mkdir("./runs")
    except FileExistsError:
        print('Folder already exist.')
        
    try:
        os.mkdir("./runs/test_" + str(1))
    except FileExistsError:
        print('Folder already exist.')

    generate_in_file(velocity, temperature, dump_dir=dump_dir, output_file=dump_dir_ + "/in.perforation", 
                     iterations=[1000, 1000, 0, 1000], ts = [0.1, 0.1, temp_ts])
    
    os.system(f'cp C60.data {dump_dir_}/C60.data')
    
    prepare_data(graphene_size=[20, 20], input_c60_file=f'{dump_dir_}/C60.data', 
                 output_c60_centered=f'{dump_dir_}/centered_C60.data', 
                 output_graphene_centered=f'{dump_dir_}/centered_graphene.data', 
                 output_file=f'{dump_dir_}/combined.data', 
                 output_graphene=f'{dump_dir_}/graphene.data', 
                 z_graphene=20, z_c60=5)

    lammps_path = "/home/korotnev/Documents/LAMMPS/lammps-29Aug2024/src/lmp_serial"  
    script_path = f'{dump_dir_}/in.perforation'

    try:
        result = subprocess.run([lammps_path, '-in', script_path], check=True, 
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"LAMMPS simulation for velocity {velocity} started successfully!")
        #print(result.stdout.decode())  
        #print(result.stderr.decode())  
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while running LAMMPS for velocity {velocity}: {e}")
        
    df_graphene = get_dataset(get_files(dump_dir_, el='gr'))
    df_c60 = get_dataset(get_files(dump_dir_, el='c60'))
    
    get_plots_coords_velocities(df_graphene, name=dump_dir_ + '/graphene')
    get_plots_coords_velocities(df_c60, name=dump_dir_ + '/c60')
    
    print(f'Фуллерен при Температуре {temperature}, скорости {velocity} : сломан : {is_broken_c60_coords(df_c60, z = 20)}')
    print(f'Графен при Температуре {temperature}, скорости {velocity} : сломан : {is_broken_graphene(df_graphene, alpha = 0.03)}')
    return is_broken_c60_coords(df_c60, z = 20)

In [14]:
def find_velocity(v_min, v_max, temp, tol = 0.1):
    print(f"Starting new velocity search with params\n temp {temp} \n v_min {v_min} \n v_max {v_max} \n")
    if 2 * (v_max - v_min) / (v_max + v_min) < tol:
        print(f"Solution has found, temp {temp}, velocity {(v_max + v_min) / 2}\n")
        return (v_max + v_min) / 2
    
    bool_ = run_lammps_simulation((v_max + v_min) / 2, temp)
    
    if bool_:
        return find_velocity(v_min, (v_max + v_min) / 2, temp, tol=tol)
    else:
        return find_velocity((v_max + v_min) / 2, v_max, temp, tol=tol)

In [15]:
def save_results_to_csv(file_path, velocity, temperature):
    data = {"Temperature": [temperature], "Velocity": [velocity]}
    df = pd.DataFrame(data)

    try:
        existing_df = pd.read_csv(file_path)
        new_df = pd.concat([existing_df, df], ignore_index=True)
    except FileNotFoundError:
        new_df = df
    new_df.to_csv(file_path, index=False)

In [16]:
def start_search(v_min, v_max, temp, tol=0.1):
    result_file = "./runs/test_" + str(test_num) +"/results.csv" 

    b_min = run_lammps_simulation(v_min, temp)
    b_max = run_lammps_simulation(v_max, temp)

    if (b_min and b_max) or (not b_min and not b_max):
        return 0

    velocity = find_velocity(v_min, v_max, temp, tol=tol)

    # Сохранение результата в CSV
    save_results_to_csv(result_file, velocity, temp)

    return velocity

In [ ]:
test_num = 1

#velocities = [0.05, 0.1, 0.5, 1.]
temperatures = [250, 275, 300, 325, 350]

with ProcessPoolExecutor() as executor:
    futures = [executor.submit(start_search, 0.01, 1.0, temperature) for temperature in temperatures]
    
    for future in futures:
        future.result()

Folder already exist.Folder already exist.Folder already exist.Folder already exist.Folder already exist.



In File generated.

Folder already exist.Folder already exist.Folder already exist.


Folder already exist.
In File generated.
In File generated.

In File generated.

Preparing data started.


In File generated.

Preparing data started.
Preparing data started.

Preparing data started.
Graphene structure with 270 atoms has been written to runs/test_1/vel_0.01_temp_300/graphene.data.



Graphene has generated. Centering data started.
Graphene structure with 270 atoms has been written to runs/test_1/vel_0.01_temp_250/graphene.data.Graphene structure with 270 atoms has been written to runs/test_1/vel_0.01_temp_325/graphene.data.
Graphene structure with 270 atoms has been written to runs/test_1/vel_0.01_temp_275/graphene.data.


Preparing data started.
Graphene has generated. Centering data started.

Graphene has generated. Centering data started.
Graphene has generated. Centering da